# 충청남도 공공데이터 개방목록

* 작성자: 송채은(2021), 김현지(2022-04)

### 수정 필요 항목
* 크롤링 시 for 문의 range 값에 해당하는 page_num 자동화 (현재 수동으로 입력해야 함)

In [1]:
from urllib.request import urlopen
from urllib import request
import requests
import json
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import json
import random
from glob import glob
import time
import datetime
from tqdm.notebook import tqdm

* http://www.chungnam.go.kr:8100/cnnet/board.do?pageNo=1&pageGNo=0&showSplitNo=10&mnu_cd=CNNMENU02498&searchCnd=0

* http://www.chungnam.go.kr:8100/cnnet/board.do?mnu_url=/cnbbs/view.do?board_seq=389463&mnu_cd=CNNMENU02498&searchCnd=0&pageNo=1&pageGNo=0&showSplitNo=10&code=609



In [2]:
today = datetime.date.today().isoformat()
today

'2022-07-17'

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [65]:
contents[3].find('a').attrs['href']

'/cnbbs/download.do?board_seq=407710&file_ord=1'

In [66]:
df_result = pd.DataFrame()
for page_num in tqdm(range(1, 16)) : #range - 페이지 번호에 따라 조정 필요

  url = f'http://www.chungnam.go.kr:8100/cnnet/board.do?pageNo={page_num}&pageGNo=0&showSplitNo=10&mnu_cd=CNNMENU02498'
  response = requests.get(url)
  html = response.text
  soup = BeautifulSoup(html, 'html.parser')

  # metadata 
  for file_num in range(1, 11) : #
    try :
      # 목록 한번에 해보자
      # 파일링크
      link = soup.select_one(f'#bbs > form > table > tbody > tr:nth-of-type({file_num}) > td.title_comm > a').attrs['href']
      page_link = 'http://www.chungnam.go.kr:8100' + link
      # 제목
      title = soup.select_one(f'#bbs > form > table > tbody > tr:nth-of-type({file_num}) > td.title_comm > a').get_text().replace('\r\n\t\t\t\t\t\t\t\t\t\t\t\xa0', '')

      # 그외 메타
      list_text = soup.select_one(f'#bbs > form > table > tbody > tr:nth-of-type({file_num})').get_text()
      # 번호
      idx_num = list_text.split('\n')[1]
      # 제공부서
      publisher_2 = list_text.split('\n')[5]
      # 작성일
      publish_date = list_text.split('\n')[6]
      # 조회수
      view = list_text.split('\n')[7]

      # 페이지 
      response_detail = requests.get(page_link)
      html_detail = response_detail.text
      soup_detail = BeautifulSoup(html_detail, 'html.parser')
      contents = soup_detail.find_all('td')
      publisher_name = contents[0].get_text()
      phone_num = contents[1].get_text()
      publisher_2_detail = contents[2].get_text()
      file_title = contents[3].get_text().split('\n')[1].replace('[내려받기]', '') # 용량은 따로 전처리 필요함
      file_url = 'http://www.chungnam.go.kr:8100' + contents[3].find('a').attrs['href']
      description = contents[4].get_text() # 텍스트 전처리 필요

      df = {'번호' : idx_num, '제목' : title, '상세페이지url' : page_link, '제공부서' : publisher_2, 
            '작성일' : publish_date, '조회수' : view, '작성자' : publisher_name,
            '연락처' : phone_num, '파일명(용량)' : file_title, '설명' : description, '다운로드url': file_url}
      df_result = df_result.append(pd.DataFrame([df]))    
    except :
      print('end')


  0%|          | 0/15 [00:00<?, ?it/s]

C:\Users\sangw\AppData\Local\Temp\ipykernel_16808\2401094153.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(pd.DataFrame([df]))
C:\Users\sangw\AppData\Local\Temp\ipykernel_16808\2401094153.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(pd.DataFrame([df]))
C:\Users\sangw\AppData\Local\Temp\ipykernel_16808\2401094153.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(pd.DataFrame([df]))
C:\Users\sangw\AppData\Local\Temp\ipykernel_16808\2401094153.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(pd.DataFrame

end
end


C:\Users\sangw\AppData\Local\Temp\ipykernel_16808\2401094153.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(pd.DataFrame([df]))


In [67]:
# 정제 
volume = []
file_name = []
df_result2 = df_result.copy()
df_result.reset_index(drop = True, inplace = True)

for num in list(df_result.index) :
  try :
    df_result.loc[ num, '파일명(용량)'].split('.csv')
    volume.append(df_result.loc[num, '파일명(용량)'].split('.csv')[1])
    file_name.append(df_result.loc[num, '파일명(용량)'].split('.csv')[0] + '.csv')
  except :
    try :
      df_result.loc[ num, '파일명(용량)'].split('.xlsx')[1]
      volume.append(df_result.loc[ num, '파일명(용량)'].split('.xlsx')[1])
      file_name.append(df_result.loc[ num, '파일명(용량)'].split('.xlsx')[0] + '.xlsx')
    except :
      volume.append('error')
      file_name.append('error')

df_result['파일명'] = file_name
df_result['용량'] = volume

# 왜 이렇게 했지..????
for i in list(df_result.index) :
    df_result.loc[i, '설명'] = df_result.loc[1, '설명'].replace('\r', '').replace('\n', '').replace('\t', '').replace('\xa0', '')

df_result['용량'] = df_result['용량'].apply(lambda a : str(a)[1:-1])
df_result.drop('파일명(용량)', axis = 1, inplace = True)

df_result = df_result.replace({'' : None})
df_result.to_csv(f'충청남도-공공데이터개방목록-{today}.csv')

In [71]:
driver = webdriver.Chrome()

In [72]:
for i in range(1,4):
    driver.get(str(df_result.loc[i,'다운로드url']))
    driver.implicitly_wait(3)

In [ ]:
for i in range(1,len(df_result)):
    driver.get(str(df_result.loc[i,'다운로드url']))
    driver.implicitly_wait(3)